# RonjaNet Code


## Importing and Loading

In [0]:
pip install git+git://github.com/fchollet/keras.git --upgrade --no-deps

In [0]:
import itertools
import multiprocessing.pool
import threading
import argparse
import glob
import json
from functools import partial

import pandas as pd
import numpy as np
import time

import keras
from keras import backend as K
from keras import layers, models
from keras import metrics
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.backend import relu, sigmoid
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.vgg16 import VGG16

import tensorflow as tf
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.lib.io import file_io
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter
import os

from sklearn.metrics import classification_report, confusion_matrix

from skimage import exposure, color
import cv2 as cv

#-------------------------------------------------------------------------------
# Base Model
#-------------------------------------------------------------------------------


def model_base(labels_dim):
    """Create a Keras Sequential model with layers."""

    model = models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(128, 128, 3)))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(labels_dim, activation='softmax', kernel_initializer='glorot_uniform'))

    model.summary()

    compile_model(model)
    return model

#-------------------------------------------------------------------------------
# VGGNet Model
#-------------------------------------------------------------------------------

def model_vgg19(labels_dim):
    
    model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))

    flatten = Flatten()
    new_layer2 = Dense(labels_dim, activation='softmax', name='my_dense_2')

    inp2 = model.input
    out2 = new_layer2(flatten(model.output))

    model2 = Model(inp2, out2)
    model2.summary(line_length=150)

    compile_model(model2)
    return model2

#-------------------------------------------------------------------------------
# RonjaNet
#-------------------------------------------------------------------------------
def model_ronja(labels_dim):
    """Create a Keras Sequential model with layers."""

    model = models.Sequential([
            
    Conv2D(128, (3,3), input_shape=(128, 128, 3), strides=2, padding='same', activation='relu'),
    Conv2D(128, (3,3), strides=2, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.15), # makes RonjaNet slower, yet helps against overfitting

    Conv2D(64, (3,3), strides=1, padding='same', activation='relu'),
    Conv2D(64, (3,3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.15),

    Conv2D(28, (3,3), strides=1, padding='same', activation='relu'),
    Conv2D(28, (3,3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.15),

    Conv2D(12, (3,3), strides=1, padding='same', activation='relu'),
    Conv2D(12, (3,3), strides=1, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.15),

    Flatten(),
    Dense(labels_dim, activation='softmax'),
    ])

    model.summary()

    compile_model(model)
    return model

#-------------------------------------------------------------------------------
# Compiling & Reading
#-------------------------------------------------------------------------------

def compile_model(model):
    opt = keras.optimizers.Adagrad(lr=0.01)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

def read_train_data():
    start_time = time.time()
    print("Start Read Train Data")
    data = np.load("trainDataSmall.npz")
    #data = np.load("/content/drive/My Drive/Colab Notebooks/trainDataSmall.npz")
    print("Train data read --- %s seconds ---" % (time.time() - start_time))
    X_train = data["X_train"]
    Y_train = data["Y_train"]
    print("Training - Total examples per class", np.sum(Y_train, axis=0))
    return [X_train, Y_train]


def read_test_data():
    start_time = time.time()
    print("Start Read Test Data")
    data = np.load("testDataSmall.npz")
    #data = np.load("/content/drive/My Drive/Colab Notebooks/testDataSmall.npz")
    print("Test data read --- %s seconds ---" % (time.time() - start_time))
    X_test = data["X_test"]
    Y_test = data["Y_test"]
    print("Testing - Total examples per class", np.sum(Y_test, axis=0))
    
    return [X_test, Y_test]


# Feed Forward Network

In [0]:
CLASS_SIZE = 5
FILE_PATH = 'checkpoint.{epoch:02d}.hdf5'
RETINOPATHY_MODEL = 'retinopathy.hdf5'

class ContinuousEval(keras.callbacks.Callback):
    """Continuous eval callback to evaluate the checkpoint once
       every so many epochs.
    """

    def __init__(self,
                 eval_frequency,
                 job_dir):
        self.eval_frequency = eval_frequency
        self.job_dir = job_dir
        [self.X_test, self.Y_test] = read_test_data()

    def on_epoch_begin(self, epoch, logs={}):
        if epoch > 0 and epoch % self.eval_frequency == 0:
            # Unhappy hack to work around h5py not being able to write to GCS.
            # Force snapshots and saves to local filesystem, then copy them over to GCS.
            model_path_glob = 'checkpoint.*'
            model_path_glob = os.path.join(self.job_dir, model_path_glob)
            checkpoints = glob.glob(model_path_glob)
            if len(checkpoints) > 0:
                checkpoints.sort()
                retinopathy_model = load_model(checkpoints[-1])
                retinopathy_model = compile_model(retinopathy_model)
                loss, acc = retinopathy_model.evaluate(
                    self.X_test, self.Y_test)
                print('\nEvaluation epoch[{}] metrics[{:.2f}, {:.2f}] {}'.format(
                    epoch, loss, acc, retinopathy_model.metrics_names))

            else:
                print('\nEvaluation epoch[{}] (no checkpoints found)'.format(epoch))

def AHE(image):
    image = exposure.equalize_adapthist(image, clip_limit=0.01)
    return image

def run():
    tf.keras.backend.clear_session()
    #local dir to write checkpoints and export model
    job_dir = 'jobdir'
    #Batch size for training steps
    train_batch_size = 500
    #Maximum number of epochs on which to train
    num_epochs = 20
    #Checkpoint per n training epochs
    checkpoint_epochs = 5
    #Perform one evaluation per n epochs
    eval_frequency = 5

    #DEFINE MODEL TO BE USED HERE
    retinopathy_model = model_ronja(CLASS_SIZE)

    try:
        os.makedirs(job_dir)
    except:
        pass

    # Unhappy hack to work around h5py not being able to write to GCS.
    # Force snapshots and saves to local filesystem, then copy them over to GCS.
    checkpoint_path = FILE_PATH
    checkpoint_path = os.path.join(job_dir, checkpoint_path)

    # Model checkpoint callback
    checkpoint = keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        verbose=2,
        period=checkpoint_epochs,
        mode='max')

    # Continuous eval callback
    evaluation = ContinuousEval(eval_frequency,
                                job_dir)

    # Tensorboard logs callback
    tblog = keras.callbacks.TensorBoard(
        log_dir=os.path.join(job_dir, 'logs'),
        histogram_freq=0,
        write_graph=True,
        embeddings_freq=0)

    callbacks = [checkpoint, evaluation, tblog]

    [X_train, Y_train] = read_train_data()
    
    datagen = ImageDataGenerator(
        #preprocessing_function=AHE,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True
        ) # ideally this should be added to for improved performance
    
    history = retinopathy_model.fit_generator(
        datagen.flow(X_train, Y_train, batch_size=train_batch_size),
        steps_per_epoch=100,
        epochs=num_epochs,
        callbacks=callbacks,
        verbose=2,
        validation_data=(evaluation.X_test, evaluation.Y_test))

    retinopathy_model.save(os.path.join(job_dir, RETINOPATHY_MODEL))

#-------------------------------------------------------------------------------
# Creating a plot for loss and accuracy for both training and validation set.
#-------------------------------------------------------------------------------

    import matplotlib.pyplot as plt
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'], 'bo')
    plt.plot(history.history['val_accuracy'], 'b')
    plt.title('Training and validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'test_acc'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'], 'bo')
    plt.plot(history.history['val_loss'], 'b')
    plt.title('Training and Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'test_loss'], loc='upper left')
    plt.show()


In [0]:

## this will download the data from the internet, it may fail if the server is not up
![ -f testDataSmall.npz ] || wget -O testDataSmall.npz "https://www.win.tue.nl/~cdecampos/testDataSmall.npz"
![ -f trainDataSmall.npz ] || wget -O trainDataSmall.npz "https://www.win.tue.nl/~cdecampos/trainDataSmall.npz"



# Run Model

In [0]:
!rm -fr jobdir/

run()

# Metrics

In [0]:
from matplotlib import pyplot as plt
from numpy import loadtxt
import numpy as np
from keras.models import load_model

model = load_model('jobdir/retinopathy.hdf5')
data = np.load("testDataSmall.npz")

X_test = data["X_test"]
Y_test = data["Y_test"]
yProbTest = model.predict_proba(X_test)
yClassTest = model.predict_classes(X_test)

In [0]:
def convert_act(yInput):
  if(yInput == [1, 0, 0, 0, 0]).all():
    return 0

  elif(yInput == [0, 1, 0, 0, 0]).all():
    return 1

  elif(yInput == [0, 0, 1, 0, 0]).all():
    return 2

  elif(yInput == [0, 0, 0, 1, 0]).all():
    return 3
    
  elif(yInput == [0, 0, 0, 0, 1]).all():
    return 4
    
val = input("Enter number of the test case: ")
imgToShow = tf.keras.preprocessing.image.array_to_img(
    X_test[int(val)],
    data_format=None,
    scale=True,
    dtype=None)

fig = plt.figure(figsize=(6,6))
ax1 = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
ax2 = plt.subplot2grid((15,9), (9,2), colspan=5, rowspan=5)

ax1.axis('off')
ax1.set_title("X_test[%s]" % (val))
ax1.imshow(imgToShow)
y_pos = np.arange(5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels([0,1,2,3,4])
ax2.set_xlabel('Probability')
ax2.invert_yaxis()
ax2.barh(y_pos, yProbTest[int(val)], xerr=0, align='center', color='green')
print("Predicted Class = %s, Actual Class = %s" % (yClassTest[int(val)], convert_act(Y_test[int(val)])))
plt.show()

In [0]:
%matplotlib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools
import matplotlib.pyplot as plt

new_Y_test = []

for x in range(len(Y_test)):
  new_Y_test.append(convert_act(Y_test[x]))

In [0]:
cm = confusion_matrix(new_Y_test, yClassTest)
print(classification_report(new_Y_test, yClassTest))

In [0]:
#https://scikit-learn.org/0.18/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
cm_plot_labels = [0, 1, 2, 3, 4]
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

#### Attempt at Preprocessing function

In [0]:
import scipy
def AHE(image):
    image = scipy.misc.toimage(image)

    # create a CLAHE object (Arguments are optional).
    lab = cv.cvtColor(image, cv.COLOR_RGB2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv.split(lab)

    #Create and apply the CLAHE
    clahe = cv.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    l_channel = clahe.apply(l_channel)
    l_channel.shape

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv.merge((l_channel, a_channel, b_channel))

    # convert iamge from LAB color model back to RGB color model
    image = cv.cvtColor(merged_channels, cv.COLOR_LAB2BGR)
    
    image = exposure.equalize_adapthist(image, clip_limit=0.01)
    return image
  

# Links


## PreProcessing
https://www.kaggle.com/tanlikesmath/training-on-previous-dataset-for-aptos

https://www.kaggle.com/taindow/pre-processing-train-and-test-images

https://www.kaggle.com/tanlikesmath/diabetic-retinopathy-resnet50-binary-cropped

https://mc.ai/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/

## Structure of CNN 
https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b

https://www.machinecurve.com/index.php/2018/12/07/convolutional-neural-networks-and-their-components-for-computer-vision/#stacking-cnn-layers-into-an-architecture